In [ ]:
!pip install medmnist

In [ ]:
import torch
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
import sklearn
from torch.utils.data import DataLoader, random_split, ConcatDataset, Subset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import os
from collections import Counter
from google.colab import drive
import concurrent.futures
import time
import random
from medmnist import BloodMNIST
import csv
import torch.nn as nn
import torch.nn.functional as F
import copy

In [ ]:
# Mount Google Drive for persistent storage
drive.mount('/content/drive')

In [ ]:
NUM_NODES = 4
NUM_GLOBAL_EPOCHS = 5
NUM_LOCAL_EPOCHS = 5
train_size = 0.9
test_size = 0.1
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_data():

  # Define the transformations
  transform = transforms.Compose([
      transforms.Resize((256, 256)),  # Resize to match the input size of the model
      transforms.ToTensor(),  # Convert images to tensor
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
  ])

  test_dataset = BloodMNIST(split="test", download=True, transform=transform)
  train_dataset = BloodMNIST(split="train", download=True, transform=transform)


  print("Data has been loaded")
  return train_dataset, test_dataset


In [ ]:
def split_data_nodes(dataset, num_nodes):
  # Determine the sizes of each split
  dataset_size = len(dataset)
  indices = list(range(dataset_size))
  np.random.shuffle(indices)

  split_sizes = np.random.randint(1, dataset_size // num_nodes + 1, size=num_nodes - 1)
  split_sizes = np.append(split_sizes, dataset_size - split_sizes.sum())
  np.random.shuffle(split_sizes)

  # Create random splits
  subsets = []
  start = 0
  for size in split_sizes:
      subset_indices = indices[start:start + size]
      subsets.append(subset_indices)
      start += size


  # Apply the splits to the dataset
  dataset_splits = [torch.utils.data.Subset(dataset, subset) for subset in subsets]

  # Calculate the size weightings for each subset
  weightings = [len(subset) / dataset_size for subset in dataset_splits]



  print("Data has been split into " + str(num_nodes) + " nodes")
  # Print the size of each data subset
  for i, subset in enumerate(dataset_splits):
      print(f'Size of subset {i+1}: {len(subset)}')


  return dataset_splits, dataset, weightings

In [ ]:
def create_data_loaders(subset):

    # Split into train and validation (adjust validation split as needed)
    val_size = int(0.2 * len(subset))
    train_size = len(subset) - val_size
    train_subset, val_subset = torch.utils.data.random_split(subset, [train_size, val_size])

    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    print("Data Loader Created")
    return train_loader, val_loader

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=256*256*3, num_classes=8):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)  # Ensure num_classes matches dataset

    def forward(self, x):
        x = x.view(-1, 256*256*3)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def create_final_data_test_loader(final_testing):
  final_model_test_loader = torch.utils.data.DataLoader(final_testing, batch_size=BATCH_SIZE, shuffle=False)
  return final_model_test_loader

In [ ]:
def model_parameters_to_string(model):
    params_str = "Model Parameters:\n"
    for name, param in model.named_parameters():
        params_str += f"Name: {name}, Shape: {param.shape}\n"
        params_str += f"{param.data}\n"
        params_str += "-" * 50 + "\n"
    return params_str

In [ ]:

class Node:
    def __init__(self, node_id, train_loader: DataLoader, test_loader: DataLoader, weighting, aggregation_methodology):
        """
        Initialize the Node with training and testing DataLoaders.

        Parameters:
        train_loader (DataLoader): DataLoader for training data.
        test_loader (DataLoader): DataLoader for testing data.
        """
        self.__train_loader = train_loader
        self.test_loader = test_loader
        self.node_id = node_id
        self.global_parameters = []
        self.accuracy = None
        self.model = SimpleNN(num_classes=8)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        self.accuracy_scores = []
        self.current_accuracy = 0
        self.accuracy_scores = []
        self.f1_scores = []
        self.precision_scores = []
        self.recall_scores = []
        self.weighting = weighting
        self.aggregation_methodology = aggregation_methodology
        self.balance_score = self.get_balance_score()

    def get_model(self):
      return self.model

    def get_balance_score(self):
        class_counts = np.zeros(10, dtype=int)  # MNIST has 10 classes (0-9)
        for _, label in self.__train_loader:
            class_counts[label] += 1
        total_samples = sum(class_counts)
        if total_samples == 0:
            return 0
        class_proportions = np.array(class_counts) / total_samples
        score = 1 - np.std(class_proportions)
        return score

    def get_final_scores(self):

      """
      Convert performance metrics lists into a pandas DataFrame.

      Parameters:
      accuracy_scores (list): List of accuracy scores.
      precision_scores (list): List of precision scores.
      recall_scores (list): List of recall scores.
      f1_scores (list): List of F1 scores.

      Returns:
      pd.DataFrame: DataFrame containing the metrics.
      """
      # Create a dictionary from the lists
      metrics_dict = {
          'Accuracy': self.accuracy_scores,
          'Precision': self.precision_scores,
          'Recall': self.recall_scores,
          'F1 Score': self.f1_scores
      }

      # Convert the dictionary into a DataFrame
      df = pd.DataFrame(metrics_dict)

      return df

    def get_accuracy_score(self):
      return self.current_accuracy

    def set_accuracy_score(self, accuracy):
      self.current_accuracy = accuracy

    def get_node_id(self):
      return self.node_id

    def set_aggregation_methodology(self, methodology):
      self.aggregation_methodology = methodology

    def get_test_loader(self) -> DataLoader:
        """
        Return the testing DataLoader.

        Returns:
        DataLoader: The testing DataLoader.
        """
        return self.test_loader

    def broadcast_parameters(self):
        """
        Return the model parameters.

        Returns:
        dict: Dictionary of model parameters.
        """
        return_list = []
        for param in self.model.parameters():
          return_list.append(param.clone())

        return_list.insert(0, self.current_accuracy)
        return_list.insert(0, self.weighting)
        return_list.insert(0, self.balance_score)

        return return_list


    def gather_parameters(self, params):
        print("Gathered Parameters for Node " + str(self.node_id))
        self.global_parameters.append(params)
        print("Length of Gathered Parameters for Node " + str(self.node_id) + " : " + str(len(self.global_parameters)))

    def size_average_parameters(self):
            averaged_params = []
            print("Size of global params: " + str(len(self.global_parameters)))

            # Create a copy of global_parameters without modifying the original
            params = [param[:] for param in self.global_parameters]  # Shallow copy of each param list
            weightings = []

            # Extract non-tensor elements without modifying the original
            for param_set in params:
                balance_score = param_set[0]  # Access without popping (no removal)
                weighting = param_set[1]      # Access weighting
                accuracy = param_set[2]       # Access accuracy

                weightings.append(weighting)

            # Clone the tensors and initialize averaged_params
            for param in params[0][3:]:  # Skip non-tensor elements (balance, weighting, accuracy)
                averaged_params.append(torch.zeros_like(param))  # Initialize with zeros

            # Sum the parameters with their corresponding weights
            total_weight = 0
            for model_params, weight in zip(params, weightings):
                model_params = model_params[3:]  # Exclude non-tensor elements
                if len(model_params) != len(averaged_params):
                    raise ValueError("Mismatch in the number of parameters between models.")

                for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
                    if param.shape != avg_param.shape:
                        raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
                    averaged_params[i] += param * weight
                total_weight += weight

            # Normalize by the total weight
            if total_weight > 0:
                for i in range(len(averaged_params)):
                    averaged_params[i] /= total_weight

            return averaged_params





    def balance_average_parameters(self):
      averaged_params = []
      print("Size of global params: " + str(len(self.global_parameters)))

      # Create a shallow copy of the global parameters to avoid mutation
      params = [param[:] for param in self.global_parameters]  # Shallow copy of each param list
      balance_scores = []

      # Extract non-tensor elements (balance scores, weightings, accuracy) without modifying the original
      for param_set in params:
          balance_score = param_set[0]  # Access balance score
          weighting = param_set[1]      # Access weighting (but not needed)
          accuracy = param_set[2]       # Access accuracy (but not needed)
          print("Balance Score: " + str(balance_score))
          balance_scores.append(balance_score)

      # Clone the tensors and initialize averaged_params with zeros
      for param in params[0][3:]:  # Skip first three elements (balance, weighting, accuracy)
          averaged_params.append(torch.zeros_like(param))  # Initialize with zeros

      # Sum the parameters with their corresponding balance scores
      total_balance_scores = 0
      for model_params, balance_score in zip(params, balance_scores):
          model_params = model_params[3:]  # Exclude the first three non-tensor elements
          if len(model_params) != len(averaged_params):
              raise ValueError("Mismatch in the number of parameters between models.")

          for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
              if param.shape != avg_param.shape:
                  raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
              averaged_params[i] += param * balance_score
          total_balance_scores += balance_score

      # Normalize by the total balance scores
      if total_balance_scores > 0:
          print("Total Balance Scores: " + str(total_balance_scores))
          for i in range(len(averaged_params)):
              averaged_params[i] /= total_balance_scores

      return averaged_params

    def test_aggregation(self):
      size_model = copy.deepcopy(self.model)  # Make a copy of the model
      balance_model = copy.deepcopy(self.model)  # Make another copy of the model
      averaged_size_params = self.size_average_parameters()
      size_state_dict = dict(zip(size_model.state_dict().keys(), averaged_size_params))
      size_model.load_state_dict(size_state_dict)

      averaged_balance_params = self.balance_average_parameters()
      balance_state_dict = dict(zip(balance_model.state_dict().keys(), averaged_balance_params))
      balance_model.load_state_dict(balance_state_dict)

      size_accuracy, size_precision, size_recall, size_f1 = evaluate(size_model, self.test_loader)
      balance_accuracy, balance_precision, balance_recall, balance_f1 = evaluate(balance_model, self.test_loader)
      print("Size Accuracy: " + str(size_accuracy))
      print("Balance Accuracy: " + str(balance_accuracy))
      if size_accuracy > balance_accuracy:
        return "size"
      else:
        return "balance"



    def aggregate_parameters(self):
      if self.aggregation_methodology == "size_average":
        averaged_params = self.size_average_parameters()
      elif self.aggregation_methodology == "balance_average":
        averaged_params = self.balance_average_parameters()
      else:
        raise ValueError("Invalid aggregation methodology. Use 'average' or 'size_average' or 'accuracy_average'.")

      print("Length of Averaged Params for node " + str(self.node_id) + " : " + str(len(averaged_params)))
      self.set_parameters(averaged_params)

      self.global_parameters = []


    def set_parameters(self, parameters):

        """
        Set the model parameters.

        Parameters:
        parameters (dict): Dictionary of model parameters.
        """
        # Convert the list of averaged parameters back into the correct format for the model
        print("Setting Parameters for node: " + str(self.node_id))
        new_state_dict = dict(zip(self.model.state_dict().keys(), parameters))
        self.model.load_state_dict(new_state_dict)
        print("Set Parameters for node " + str(self.node_id) + " : " + str(model_parameters_to_string(self.model)))



    def train(self):

      # Move model to the specified device
      self.model.to(device)

      for epoch in range(NUM_LOCAL_EPOCHS):

        self.model.train()
        running_loss = 0.0

        for images, labels in self.__train_loader:
            # Move images and labels to the specified device
            images, labels = images.to(device), labels.to(device)

            # Flatten the labels tensor to be 1D
            if len(labels.shape) == 2:
                labels = labels.squeeze(1)  # Remove the extra dimension

            # Check for label values out of bounds
            if labels.max() >= 8:
                raise ValueError(f"Label value {labels.max()} is out of bounds for the number of classes {8}.")

            # Ensure labels are 1D
            if len(labels.shape) != 1:
                raise ValueError("Labels tensor is not 1D. It should be of shape [batch_size].")

            self.optimizer.zero_grad()
            outputs = self.model(images)

            # Ensure labels are of type long
            labels = labels.long()

            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
      # Print the average loss for this epoch
        print(f"Epoch [{epoch+1}/{NUM_LOCAL_EPOCHS}], Training Loss: {running_loss / len(train_loader)}")


    def evaluate(self):
        self.model.eval()  # Set the model to evaluation mode

        all_labels = []
        all_predictions = []

        with torch.no_grad():  # No need to calculate gradients during evaluation
            for images, labels in self.test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        # Convert lists to numpy arrays for sklearn metrics
        all_labels = np.array(all_labels).flatten()
        all_predictions = np.array(all_predictions)

        # Calculate metrics
        accuracy = np.sum(all_predictions == all_labels) / len(all_labels)
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')
        f1 = f1_score(all_labels, all_predictions, average='weighted')

        self.accuracy_scores.append(accuracy)
        self.precision_scores.append(precision)
        self.recall_scores.append(recall)
        self.f1_scores.append(f1)
        self.set_accuracy_score(accuracy)
        return accuracy, precision, recall, f1





In [ ]:
def train_and_communicate(node, nodes):
    node.train()
    metrics = node.evaluate()
    print("Metrics for Node " + str(node.get_node_id()) + " : " + str(metrics))
    # Send parameters to all other nodes
    for other_node in nodes:
      print("NODE " + str(node.get_node_id()) + "  HAS BROADCASTED ")

      other_node.gather_parameters(node.broadcast_parameters())

In [ ]:
def dynamic_aggregation(nodes):
  decisions = []
  for node in nodes:
    decisions.append(node.test_aggregation())
  if decisions.count("size") > decisions.count("balance"):
    for node in nodes:
      node.set_aggregation_methodology("size_average")
    return "balance"
  elif decisions.count("size") < decisions.count("balance"):
    for node in nodes:
      node.set_aggregation_methodology("balance_average")
    return "size"
  else:
    for node in nodes:
      node.set_aggregation_methodology("balance_average")

    return "size"

In [ ]:
def evaluate(model, test_loader):
  model.eval()  # Set the model to evaluation mode

  all_labels = []
  all_predictions = []

  with torch.no_grad():  # No need to calculate gradients during evaluation
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

  # Convert lists to numpy arrays for sklearn metrics
  all_labels = np.array(all_labels).flatten()
  all_predictions = np.array(all_predictions)

    # Print the shapes of the arrays
  print(f"Shape of all_labels: {all_labels.shape}")
  print(f"Shape of all_predictions: {all_predictions.shape}")


  print(f"Unique labels: {np.unique(all_labels)}")
  print(f"Unique predictions: {np.unique(all_predictions)}")


  print(f"Sample Labels: {all_labels[:10]}")
  print(f"Sample Predictions: {all_predictions[:10]}")

  print("Num Correct: " + str(np.sum(all_predictions == all_labels)))

  # Calculate metrics
  accuracy = np.sum(all_predictions == all_labels) / len(all_labels)
  precision = precision_score(all_labels, all_predictions, average='weighted')
  recall = recall_score(all_labels, all_predictions, average='weighted')
  f1 = f1_score(all_labels, all_predictions, average='weighted')

  return accuracy, precision, recall, f1

In [ ]:
# Function to write the metrics to a CSV file
def write_metrics_to_csv(filename, metrics_list):
    file_exists = os.path.isfile(filename)

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        # Write the header only if the file is being created
        if not file_exists:
            writer.writerow(['Aggregation Method', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

        # Write the data
        writer.writerow(metrics_list)

    print(f'Metrics written to {filename}')

In [ ]:
for i in range(10):
    print("Swarm Number: " + str(i))
    dataset, testing_set = load_data()

    #split data into final testing and data for nodes
    subsets, dataset, weightings = split_data_nodes(dataset, NUM_NODES)

    #create the final test loader and set it
    final_test_loader = create_final_data_test_loader(testing_set)

    nodes = []
    for i in range(0,NUM_NODES):
      train_loader, val_loader =  create_data_loaders(subsets[i])
      node = Node(i, train_loader, val_loader, weightings[i], "balance_average")
      nodes.append(node)

    winners = []
    # Training and aggregation process
    for _ in range(NUM_GLOBAL_EPOCHS):  # Number of training rounds
        print("Round " + str(_))
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(train_and_communicate, nodes, [nodes]*len(nodes))

        if _ < NUM_GLOBAL_EPOCHS / 2:
            winner = dynamic_aggregation(nodes)
            winners.append(winner)
            print("Winners: " + str(winners))
        else:
            winner = max(set(winners), key=winners.count)
            print("Winner: " + str(winner))
            if winner == "size":
                for node in nodes:
                    node.set_aggregation_methodology("size_average")
            elif winner == "balance":
                for node in nodes:
                    node.set_aggregation_methodology("balance_average")
            else:
              print("MAJOR ERROR: Winner is in fact: " + str(winner))

        # Aggregation phase
        for node in nodes:

          updated_params = node.aggregate_parameters()

        time.sleep(1)  # Simulate time between rounds


    node = nodes[0]

    accuracy, precision, recall, f1 = evaluate(node.get_model(), final_test_loader)
    filename = '/content/drive/My Drive/Swarm_Learning/dynamic_bloodMNIST.csv'
    metrics_list = [accuracy, precision, recall, f1]
    write_metrics_to_csv(filename, metrics_list)



